In [1]:
# код из ДЗ1

In [2]:
import random
import itertools

import pandas as pd
import numpy as np
import scipy.sparse
from scipy.sparse import csr_matrix, csc_matrix

def preprocess_inplace(data: pd.DataFrame):
    data['date_time'] = pd.to_datetime(data['date_time'])
    data.sort_values(by='date_time', inplace=True)
    

from typing import List, Tuple, Union, Optional, Dict

Feature = Tuple[List[str], np.array]
CategoricalFeature = Tuple[List[str], csc_matrix]

def categorical_feature_from_series(series: pd.Series) -> CategoricalFeature:
    dummies = pd.get_dummies(series, prefix=series.name, sparse=True)
    return dummies.columns.tolist(), csc_matrix(dummies.sparse.to_coo())        

def product_of_categorical_features(
    feature1: CategoricalFeature, feature2: CategoricalFeature
) -> CategoricalFeature:
    names1, feature1 = feature1
    names2, feature2 = feature2
    
    new_names = []
    new_columns = []
    for i, name1 in enumerate(names1):
        for j, name2 in enumerate(names2):
            new_names.append(f"{name1}__{name2}")
            new_columns.append(feature1[:, i].multiply(feature2[:, j]))
    
    return new_names, scipy.sparse.hstack(new_columns)

def feature_engineering(data: pd.DataFrame, feature_products: List[Tuple[str, str]] = []) -> Feature:
    cat_feature_names = ["zone_id", "banner_id", "os_id", "country_id"]
    features = [categorical_feature_from_series(data[name]) for name in cat_feature_names]
    
    cat_feature_dict = dict(zip(cat_feature_names, features))
    for name1, name2 in feature_products:
        product_feature = product_of_categorical_features(cat_feature_dict[name1], cat_feature_dict[name2])
        features.append(product_feature)
    
    # features.append((["log_clicks"], csc_matrix(np.log(1 + data.campaign_clicks.values.reshape(-1, 1)))))
    
    # gather everything
    all_names, all_features = [], []
    for names, csc in features:
        all_names.extend(names)
        all_features.append(csc)
    all_features = csr_matrix(scipy.sparse.hstack(all_features))
    
    return all_names, all_features

# %time basic_features = feature_engineering(data)

In [3]:
def last_day_eval_split(data, X, y):
    last_event = data.date_time.iloc[-1]
    day = last_event.day
    month = last_event.month
    year = last_event.year
    events_on_last_day = ((data.date_time.dt.day == day) & (data.date_time.dt.month == month) & (data.date_time.dt.year == year)).sum()
    
    X_train = X[:-events_on_last_day]
    y_train = y[:-events_on_last_day]
    
    X_eval = X[-events_on_last_day:]
    y_eval = y[-events_on_last_day:]
    
    data_train = data.iloc[:-events_on_last_day]
    data_eval = data.iloc[-events_on_last_day:]
    
    return data_train, data_eval, X_train, y_train, X_eval, y_eval
    
def train_val_split(data, X, y):
    n = X.shape[0]
    
    val = n // 10
    
    X_train = X[:-val]
    X_val = X[-val:]
    
    y_train = y[:-val]
    y_val = y[-val:]
    
    data_train = data.iloc[:-val]
    data_val = data.iloc[-val:]
    
    return data_train, data_val, X_train, y_train, X_val, y_val

In [4]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss

def create_model(X, y, C=1.0):
    y = y.astype(np.float64).todense().reshape((-1, 1))  # no idea why
    model = LogisticRegression(penalty='l2', C=C, solver='lbfgs')
    model.fit(X, y)
    return model

In [5]:
# фильтруем чтобы суммы в уравнениях имели смысл (не можем применить модель к banner_id0 и banner_id1 которых не было на обучении)

In [6]:
def filter_dataframe_and_features_for_cips(data, features):
    bannerid_equals_bannerid0 = (data['banner_id'] == data['banner_id0'])
    banners_in_train_set = set(data['banner_id'])
    have_coefficients_for_banners_in_train_set = data['banner_id1'].map(lambda i: i in banners_in_train_set)
    
    data_for_cips = data[bannerid_equals_bannerid0 & have_coefficients_for_banners_in_train_set].copy()
    features_for_cips = features[bannerid_equals_bannerid0 & have_coefficients_for_banners_in_train_set].copy()
    
    return data_for_cips, features_for_cips

In [7]:
# %data_for_cips, features_for_cips = filter_dataframe_and_features_for_cips(data, features)

In [8]:
from scipy.special import logit
from scipy.sparse import lil_matrix
from scipy.stats import norm


def bannerid_to_bannerid1(data: pd.DataFrame, features: csc_matrix, feature_names: List[str]) -> lil_matrix:
    banner_id_to_feature_id = {}
    for idx, name in enumerate(feature_names):
        if name.startswith("banner_id"):
            _, banner_id = name.rsplit("_", maxsplit=1)
            banner_id_to_feature_id[int(banner_id)] = idx
    
    n, _ = features.shape
    
    banner_pairs: List[Dict[str, int]] = data[['banner_id', 'banner_id1']].to_dict('records')
    features_bid1: lil_matrix = features.tolil()
    for row_idx, row in enumerate(banner_pairs):
        to_unset = banner_id_to_feature_id[row['banner_id']]
        to_set = banner_id_to_feature_id[row['banner_id1']]
        features_bid1[row_idx, to_unset] = 0.0
        features_bid1[row_idx, to_set] = 1.0
        
    return features_bid1


def normal1_greater_than_normal2(mu1, sigma1, mu2, sigma2):
    mu_diff = mu1 - mu2
    sigma_diff = np.sqrt(sigma1 ** 2 + sigma2 ** 2)
    return 1 - norm.cdf(0, loc=mu_diff, scale=sigma_diff)


def cips(
    data: pd.DataFrame,
    features_bid0: csc_matrix,
    features_bid1: lil_matrix,
    model: LogisticRegression,
    lambda_: float = 10.
):
    # по условию задания
    coeff_sum0_new = logit(model.predict_proba(features_bid0)[:, -1])
    coeff_sum1_new = logit(model.predict_proba(features_bid1)[:, -1])
    
    # "старая" policy
    pi0 = normal1_greater_than_normal2(
        mu1=data.coeff_sum0,
        sigma1=data.g0,
        mu2=data.coeff_sum1,
        sigma2=data.g1
    )
    
    # новая policy
    pi1 = normal1_greater_than_normal2(
        mu1=coeff_sum0_new,
        sigma1=data.g0,
        mu2=coeff_sum1_new,
        sigma2=data.g1
    )
    
    reward = data.clicks.values
    clipped_ips = np.nanmean(
        np.minimum(pi1 / pi0, lambda_) * reward
    )
    
    return clipped_ips

In [9]:
# "большая" функция которая делает всё
def calculate_cips(data, features, feature_names, model):
    data_for_cips, features_for_cips = filter_dataframe_and_features_for_cips(data, features)
    features_for_cips_bid1 = bannerid_to_bannerid1(data_for_cips, features_for_cips, feature_names)
    return cips(
        data=data_for_cips,
        features_bid0=features_for_cips,
        features_bid1=features_for_cips_bid1,
        model=model
    )

### Подбираем C опираясь на Clipped IPS в качестве метрики (в первом дз использовался log loss на последнем дне)

Off-Policy Learning: $\hat{\pi} = \arg \max_{\pi \in \Pi} \hat{V}_{CIPS}(\pi;D_0, \lambda=10)$

In [10]:
def cv(data, features, feature_names, answers):
    models = {}
    
    val_cipses = {}
    
    C_grid = [0.000001, 0.000005, 0.00001, 0.00005, 0.0001, 0.0005, 0.001, 0.005, 0.01]  # larger do not converge
    data_, data_eval, X, y, X_eval, y_eval = last_day_eval_split(data, features, answers)
    data_train, data_val, X_train, y_train, X_val, y_val = train_val_split(data_, X, y)

    for C in C_grid:
        model = create_model(X_train, y_train, C)
        
        val_loss = log_loss(y_val.todense(), model.predict_proba(X_val))
        val_cips = calculate_cips(data_val, X_val, feature_names, model)
        
        models[C] = model
        val_cipses[C] = val_cips 
        
        print(f"C={C:.6f}\tval_logloss={val_loss:.6f}\tval_CIPS={val_cips:.6f}")
        
    best_cips = -1
    best_model = None
    best_model_C = None
    for C, cips_ in val_cipses.items():
        if cips_ > best_cips:
            best_cips = cips_
            best_model = models[C]
            best_C = C
    
    lastday_logloss = log_loss(y_eval.todense(), best_model.predict_proba(X_eval))
    lastday_cips = eval_cips = calculate_cips(data_eval, X_eval, feature_names, best_model)
    
    return {
        "model": best_model,
        "lastday_cips": lastday_cips,
        "lastday_logloss": lastday_logloss,
        "C": best_C
    }

In [11]:
data = pd.read_csv('../data/data.csv')
preprocess_inplace(data)

answers = csr_matrix(data.clicks.values.reshape(-1, 1))
%time names, features = feature_engineering(data, feature_products=[["os_id", "country_id"]])

CPU times: user 15.5 s, sys: 832 ms, total: 16.3 s
Wall time: 16.3 s


In [12]:
import warnings
from sklearn.exceptions import DataConversionWarning

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    warnings.warn("", RuntimeWarning)
    warnings.warn("", FutureWarning)
    warnings.warn("", DataConversionWarning)
    
    cv_result = cv(data, features, names, answers)
    
    print(cv_result)

C=0.000001	val_logloss=0.185062	val_CIPS=0.065711
C=0.000005	val_logloss=0.182732	val_CIPS=0.066546
C=0.000010	val_logloss=0.180679	val_CIPS=0.066604
C=0.000050	val_logloss=0.173228	val_CIPS=0.062075
C=0.000100	val_logloss=0.169614	val_CIPS=0.059645
C=0.000500	val_logloss=0.163407	val_CIPS=0.059128
C=0.001000	val_logloss=0.161664	val_CIPS=0.058043
C=0.005000	val_logloss=0.159087	val_CIPS=0.059951
C=0.010000	val_logloss=0.158519	val_CIPS=0.058464
{'model': LogisticRegression(C=1e-05), 'lastday_cips': 0.045828518319138006, 'lastday_logloss': 0.15187974190966094, 'C': 1e-05}
